In [ ]:
# TUTORIAL FROM https://colab.research.google.com/drive/17Fql7pyK3xsO8KmZorvb1tBoPomidCPB#scrollTo=hKDhJGQVqb3Y 
# enables music21 to render images of musical notes
print('installing lilypond...')
!apt-get install lilypond > /dev/null
# converts midi files to wav files into order to play them
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
print('done!')

installing lilypond...
Extracting templates from packages: 100%
installing fluidsynth...
done!


In [ ]:
def show(music):
  display(Image(str(music.write('lily.png'))))

def play(music):
  filename = music.write('mid')
  !fluidsynth -ni font.sf2 $filename -F $filename\.wav -r 16000 > /dev/null
  display(Audio(filename + '.wav'))

In [ ]:
import music21 
from music21 import *
# music21.configure.run()

import glob
import pickle
from IPython.display import Image, Audio # to show scores and play audios
import os # to join paths (folders and file names)
import csv
import copy
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# this should enable MuseScore to open up when needed and show scores, but it does not work
'''
us = music21.environment.UserSettings()
us['musicxmlPath'] = '/Applications/MuseScore 3.app/Contents/MacOS/mscore'
us['musescoreDirectPNGPath'] = '/Applications/MuseScore 3.app/Contents/MacOS/mscore'
'''

"\nus = music21.environment.UserSettings()\nus['musicxmlPath'] = '/Applications/MuseScore 3.app/Contents/MacOS/mscore'\nus['musescoreDirectPNGPath'] = '/Applications/MuseScore 3.app/Contents/MacOS/mscore'\n"

In [ ]:
# GLOBAL PATHS
jingJuScoresDataset_XMLFolder = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Computational Musicology/Assignment/Jingju Scores Dataset/MusicXML'
linesCSV_FilePath = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Computational Musicology/Assignment/Jingju Scores Dataset/MuseScore/lines_data.csv'
linesSimilarityCSV_FilePath = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Computational Musicology/Assignment/lines_similarity.csv'
pickleFilesFolder = '/content/gdrive/MyDrive/Colab Notebooks/Audio and Music Processing Lab/Computational Musicology/Assignment'

In [ ]:
'''
def getListOfHeterophonyIntervals_Between2Parts(voicePart, accompPart):
  # This function takes 2 parts (voice and accompaniment part) and computes the 
  # heterohpny intervals (relative to the voice part, excluding unisons)
  # between them. Returns a list of dictionaries
  # (one dictionary for each heterophony interval) with;
  # {'Interval_class':, 'Interval_duration':, 'Voice_part_lyrics':})

  listOfIntervals = list()
  soundingNotes = music21.stream.Stream()
  for voiceNote in voicePart.notes:
    newSoundingNotes = accompPart.getElementsByOffset(voiceNote.offset, offsetEnd=voiceNote.offset+voiceNote.quarterLength, includeElementsThatEndAtStart=False, mustFinishInSpan=False, mustBeginInSpan=False, includeEndBoundary=False).stream()
    for accompNote in newSoundingNotes.notes:      
      interval = music21.interval.Interval(noteStart = voiceNote, noteEnd = accompNote)
      if interval.semitones != 0: # if interval is not a unison
        if accompNote.offset < voiceNote.offset: # if accompNote starts earlier than voiceNote
          interval.quarterLength = (accompNote.offset + accompNote.quarterLength) - voiceNote.offset
          interval.offset = voiceNote.offset
        else:
          interval.quarterLength = accompNote.quarterLength
          interval.offset = accompNote.offset
        # if accompaniment note ends after voice note's end 
        if (accompNote.offset + accompNote.quarterLength) > (voiceNote.offset + voiceNote.quarterLength):
          interval.quarterLength = ((accompNote.offset + accompNote.quarterLength) - (voiceNote.offset + voiceNote.quarterLength)) - accompNote.offset 

        duration = music21.duration.Duration()
        duration.quarterLength = interval.quarterLength

        if voiceNote.hasLyrics:
          listOfIntervals.append({'Measure_number': voiceNote.measureNumber, 'Interval_class': interval.directedNiceName, 'Interval_duration': interval.quarterLength, 'Voice_part_lyrics': voiceNote.lyric})
        else:
          listOfIntervals.append({'Measure_number': voiceNote.measureNumber, 'Interval_class': interval.directedNiceName, 'Interval_duration': interval.quarterLength})

  return listOfIntervals
'''

"\ndef getListOfHeterophonyIntervals_Between2Parts(voicePart, accompPart):\n  # This function takes 2 parts (voice and accompaniment part) and computes the \n  # heterohpny intervals (relative to the voice part, excluding unisons)\n  # between them. Returns a list of dictionaries\n  # (one dictionary for each heterophony interval) with;\n  # {'Interval_class':, 'Interval_duration':, 'Voice_part_lyrics':})\n\n  listOfIntervals = list()\n  soundingNotes = music21.stream.Stream()\n  for voiceNote in voicePart.notes:\n    newSoundingNotes = accompPart.getElementsByOffset(voiceNote.offset, offsetEnd=voiceNote.offset+voiceNote.quarterLength, includeElementsThatEndAtStart=False, mustFinishInSpan=False, mustBeginInSpan=False, includeEndBoundary=False).stream()\n    for accompNote in newSoundingNotes.notes:      \n      interval = music21.interval.Interval(noteStart = voiceNote, noteEnd = accompNote)\n      if interval.semitones != 0: # if interval is not a unison\n        if accompNote.offset 

In [ ]:
'''
def filter_FullHeterophonyIntervalsList_IntoLineIntervalsDict(fullHeterophonyIntervalsList, lineDuration, startMeasure, endMeasure):
  lineIntervalsList = dict()
  for intervalDict in fullHeterophonyIntervalsList:
    if intervalDict['Measure_number'] >= startMeasure and intervalDict['Measure_number'] <= endMeasure:
      if intervalDict['Interval_class'] in lineIntervalsList:
        lineIntervalsList[intervalDict['Interval_class']] = lineIntervalsList[intervalDict['Interval_class']] + intervalDict['Interval_duration']
      else:
        lineIntervalsList[intervalDict['Interval_class']] = intervalDict['Interval_duration']
      lineIntervalsList[intervalDict['Interval_class']] = round(lineIntervalsList[intervalDict['Interval_class']] / lineDuration, 3)
    # print(lineIntervalsList)
  return lineIntervalsList
'''

"\ndef filter_FullHeterophonyIntervalsList_IntoLineIntervalsDict(fullHeterophonyIntervalsList, lineDuration, startMeasure, endMeasure):\n  lineIntervalsList = dict()\n  for intervalDict in fullHeterophonyIntervalsList:\n    if intervalDict['Measure_number'] >= startMeasure and intervalDict['Measure_number'] <= endMeasure:\n      if intervalDict['Interval_class'] in lineIntervalsList:\n        lineIntervalsList[intervalDict['Interval_class']] = lineIntervalsList[intervalDict['Interval_class']] + intervalDict['Interval_duration']\n      else:\n        lineIntervalsList[intervalDict['Interval_class']] = intervalDict['Interval_duration']\n      lineIntervalsList[intervalDict['Interval_class']] = round(lineIntervalsList[intervalDict['Interval_class']] / lineDuration, 3)\n    # print(lineIntervalsList)\n  return lineIntervalsList\n"

In [ ]:
'''
listOfUniqueIntervals = ['Descending Minor Third', 'Ascending Perfect Octave', 'Descending Minor Second', 'Ascending Perfect Fifth', 'Ascending Minor Seventh', 'Descending Major Second', 'Descending Perfect Eleventh', 'Ascending Minor Second', 'Descending Perfect Octave', 'Ascending Perfect Fourth', 'Descending Perfect Twelfth', 'Descending Perfect Fifth', 'Line_number:', 'Descending Minor Tenth', 'Descending Minor Ninth', 'Descending Major Seventh', 'Descending Major Ninth', 'Ascending Major Second', 'Ascending Minor Third', 'Descending Major Tenth', 'Descending Minor Seventh', 'Ascending Major Sixth', 'Descending Minor Sixth', 'Descending Major Third', 'Ascending Major Third', 'Descending Major Sixth', 'Ascending Minor Sixth', 'Descending Perfect Fourth']
print(listOfUniqueIntervals)
'''

"\nlistOfUniqueIntervals = ['Descending Minor Third', 'Ascending Perfect Octave', 'Descending Minor Second', 'Ascending Perfect Fifth', 'Ascending Minor Seventh', 'Descending Major Second', 'Descending Perfect Eleventh', 'Ascending Minor Second', 'Descending Perfect Octave', 'Ascending Perfect Fourth', 'Descending Perfect Twelfth', 'Descending Perfect Fifth', 'Line_number:', 'Descending Minor Tenth', 'Descending Minor Ninth', 'Descending Major Seventh', 'Descending Major Ninth', 'Ascending Major Second', 'Ascending Minor Third', 'Descending Major Tenth', 'Descending Minor Seventh', 'Ascending Major Sixth', 'Descending Minor Sixth', 'Descending Major Third', 'Ascending Major Third', 'Descending Major Sixth', 'Ascending Minor Sixth', 'Descending Perfect Fourth']\nprint(listOfUniqueIntervals)\n"

In [ ]:
def calculateSimilarity(voice_flat, accom_flat):
  """
  Given two flattened Music21 score parts, it will calculate what percentage of the voice's total
  performance has rhythmic or melodic overlap with the accompanying part. Pitch is defined as having
  the exact same pitch (same octave), and rhythm must be the same start and end time. 
  It returns these two values as a percentage, i.e. if the melody plays 4 quarter-notes, and the 
  accompaniament plays one quarter note of the same pitch, it will be a ratio of 25%.

  Inputs:
  voice_flat (flattened music21 part), the 'melodic' line
  accom_flat (flattened music21 part), the 'accompaniament' line
  
  Returns:
  Pitch Ratio (float)
  Rhythm Ratio (float)
  """
  total_voice_notes = 0
  total_voice_rhythm = 0.0
  p_sim = 0.0
  r_sim = 0.0
  
  for voice_note in voice_flat.notes:
    #remove grace notes
    if voice_note.duration.quarterLength > 0.0: 
      total_voice_notes += 1
      total_voice_rhythm += voice_note.duration.quarterLength

      # define the range of the melodic note, and get all accompaniament notes in that range
      start_range = voice_note.offset
      end_range = voice_note.offset + voice_note.duration.quarterLength
      accompany_filtered = copy.deepcopy(accom_flat)

      notes_in_range = [n for n in accompany_filtered.getElementsByOffset
                        (start_range, end_range, 
                        includeEndBoundary=False,
                        mustBeginInSpan=True) 
      if isinstance(n, note.Note)]


      if not notes_in_range:
        pass

      else:
        
        # Rhythm is only considered identical if there is a single note with the same start/end offsets as voice note
        if len(notes_in_range) == 1:
          single_note = notes_in_range[0]
          if single_note.duration.quarterLength == voice_note.duration.quarterLength:
            r_sim += voice_note.duration.quarterLength

        # For each overlapping accom note, we check if it is the same as voice note. If so, we add its duration
        # and calculate the total as a ratio to the duration of the voice note
        for one_note in notes_in_range:
          if one_note.pitch.nameWithOctave == voice_note.pitch.nameWithOctave:
            p_sim += min(one_note.duration.quarterLength, voice_note.duration.quarterLength)

  if p_sim > 0.0:
    pitch_ratio = round((p_sim / total_voice_rhythm * 100), 2)
  else:
    pitch_ratio = 0.0
  
  if r_sim > 0.0:
    rhythm_ratio = round((r_sim / total_voice_rhythm * 100), 2)
  else:
    rhythm_ratio = 0.0

  return pitch_ratio, rhythm_ratio

In [ ]:


def getSimilarityBetweenOffsets_FromXMLFile(xmlFile, startOffset, endOffset):
  # print(f'Called getSimilarityBetweenOffsets_FromXMLFile, startOffset ; {startOffset} , endOffset ; {endOffset}')
  
  s = converter.parse(xmlFile)
  full_song = s.parts.stream()

  thisFile_Dict = dict()

  wasAccompPartFound = False
  wasVoicePartAFound = False

  accompPartNum = 0
  voicePartANum = 0

  parts = copy.deepcopy(full_song)

  # Part docs
  # http://web.mit.edu/music21/doc/moduleReference/moduleStreamBase.html#module-music21.stream.base
  count = 0
  for part in parts:
    if len(part.lyrics()) == 0 and wasAccompPartFound == False:
      accom_flattened = part.flat.notes.stream()
      accompPart = accom_flattened.getElementsByOffset(startOffset, endOffset)
      wasAccompPartFound = True
      accompPartNum = count
    elif len(part.lyrics()) > 0 and wasVoicePartAFound == False:
      voice_flattened = part.flat.notes.stream()
      voicePartA = voice_flattened.getElementsByOffset(startOffset, endOffset)
      wasVoicePartAFound = True
      voicePartANum = count
    else:
      break
    count += 1

  if wasVoicePartAFound and wasAccompPartFound:
    pitch_similarity, rhythm_similarity = calculateSimilarity(voicePartA, accompPart)
    thisFile_Dict = {'Pitch_similarity': pitch_similarity, 'Rhythm_similarity': rhythm_similarity}

  # print(f'Calculated similarity ; {thisFile_Dict}')
  return thisFile_Dict

In [ ]:
with open(linesCSV_FilePath, 'r') as lineCSV_File:
  with open(linesSimilarityCSV_FilePath, "w") as linesSimilarityCSV_File:
    fieldnames = ['File name', 'Role type', 'Shengqiang', 'Banshi', 'Couplet line', 'Lyrics line', 'Line_number', 'Start', 'End', 'Pitch_similarity', 'Rhythm_similarity']
    csvWriter = csv.DictWriter(linesSimilarityCSV_File, fieldnames=fieldnames)
    csvWriter.writeheader()
    reader = csv.DictReader(lineCSV_File)
    lastFileNameAnalyzed = str()
    lineCount = 0
    for row in reader:
        if row['File name'] != lastFileNameAnalyzed:
          lastFileNameAnalyzed = row['File name']
          lineCount = 0
        if os.path.isfile(os.path.join(jingJuScoresDataset_XMLFolder, row['File name'])):
          if '/' not in row['Start'] and '/' not in row['End'] and '.' not in row['Start'] and '.' not in row['End']:
            startOffset = row['Start']
            startOffset = startOffset.replace(' ', '')
            startOffset = startOffset.replace(',', '.')
            endOffset = row['End']
            endOffset = endOffset.replace(' ', '')
            endOffset = endOffset.replace(',', '.')
            if startOffset != '' and endOffset != '':
              rowDict = {'File name': row['File name'], 'Role type': row['Role type'], 'Shengqiang': row['Shengqiang'], 'Banshi': row['Banshi'], 'Couplet line': row['Couplet line'], 'Lyrics line': row['Lyrics line'], 'Line_number': lineCount+1, 'Start': float(startOffset), 'End': float(endOffset)}
              rowDict.update(getSimilarityBetweenOffsets_FromXMLFile(os.path.join(jingJuScoresDataset_XMLFolder, rowDict['File name']), rowDict['Start'], rowDict['End']))
              csvWriter.writerow(rowDict)
              lineCount += 1